Click to Edit Text

In [ ]:
# Brief check for using GPU
import torch

print(torch.cuda.is_available())


print(torch.cuda.current_device())


print(torch.cuda.device(0))


print(torch.cuda.device_count())


print(torch.cuda.get_device_name(0))

True
0
1
Tesla K80


In [ ]:
# necessary imports
import numpy as np
import csv
import os
from google.colab import drive 
import matplotlib.pyplot as plt
import random # Added by Caleb
import math # Added by Caleb
import os
import torch
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

print("Sucessfully imported all of the necessary libraries!")

Sucessfully imported all of the necessary libraries!


In [ ]:
 drive.mount('/content/drive/') ## mount to drive. This will ask for permission to access your Google drive each time

Mounted at /content/drive/


In [ ]:
data_folder_path = "/content/drive/Shareddrives/Neural Nets/Competition/ECE542_sp2022_Project_TerrainRecognition/" # path into Lobton's directory 
type_of_data = "TrainingData" # Read in the type of data you want. Options are either:  'TrainingData' or 'TestData'
list_of_files = os.listdir(data_folder_path + type_of_data) # List everything in the directory at place 2022_Project_TerrainRecognition/TrainingData or /TestData (from line above)
list_of_files.sort() # Sort the list of files
print(list_of_files) # Print out the list of files
# need to load all this data in for augmentation (only the x data, but need to match what y data it connects with)
# we need to figure out what data we want to use too. 

['subject_001_01__x.csv', 'subject_001_01__x_time.csv', 'subject_001_01__y.csv', 'subject_001_01__y_time.csv', 'subject_001_02__x.csv', 'subject_001_02__x_time.csv', 'subject_001_02__y.csv', 'subject_001_02__y_time.csv', 'subject_001_03__x.csv', 'subject_001_03__x_time.csv', 'subject_001_03__y.csv', 'subject_001_03__y_time.csv', 'subject_001_04__x.csv', 'subject_001_04__x_time.csv', 'subject_001_04__y.csv', 'subject_001_04__y_time.csv', 'subject_001_05__x.csv', 'subject_001_05__x_time.csv', 'subject_001_05__y.csv', 'subject_001_05__y_time.csv', 'subject_001_06__x.csv', 'subject_001_06__x_time.csv', 'subject_001_06__y.csv', 'subject_001_06__y_time.csv', 'subject_001_07__x.csv', 'subject_001_07__x_time.csv', 'subject_001_07__y.csv', 'subject_001_07__y_time.csv', 'subject_001_08__x.csv', 'subject_001_08__x_time.csv', 'subject_001_08__y.csv', 'subject_001_08__y_time.csv', 'subject_002_01__x.csv', 'subject_002_01__x_time.csv', 'subject_002_01__y.csv', 'subject_002_01__y_time.csv', 'subject_

This next code block creates a session class

In [ ]:
# Code block created by Caleb
# Goal: Session class that holds subject and session number, and also returns the data for that session

# This class can return the name of the file that holds the requested data
    # Example of how to use:
    # newSession = Session(4, 5)
    # newStr = newSession.xTimeName()
    # print(newStr)
    # # Prints out "subject_004_05__x.csv"

class Session:
  def __init__(self, subject_number, session_number):
    self.subject_number = subject_number  # Initialize subject number
    self.session_number = session_number  # Initialize session number

  # Each of the following member functions return the name of the specified file for that subject and session number of the Session:

  def xTimeName(self):
    return "subject_00" + str(self.subject_number) + "_0" + str(self.session_number) + "__x_time.csv"

  def yTimeName(self):
    return "subject_00" + str(self.subject_number) + "_0" + str(self.session_number) + "__y_time.csv"

  def xDataName(self):
    return "subject_00" + str(self.subject_number) + "_0" + str(self.session_number) + "__x.csv"

  def yDataName(self):
    return "subject_00" + str(self.subject_number) + "_0" + str(self.session_number) + "__y.csv"


  # This function input is the Session object that contians the subject and session numbers
  def getXDataFromFile(self):
    x_data_path = data_folder_path + "TrainingData/" + self.xDataName() # Get the path to the x_data file 
    x_data = np.genfromtxt(x_data_path, delimiter=',')  # Read the data in from the text file
    return x_data  # Return the data array
    # This function returns the array of all the six x values


  def getYDataFromFile(self):
    y_data_path = data_folder_path + "TrainingData/" + self.yDataName() # Get the path to the x_data file 
    y_data = np.genfromtxt(y_data_path, delimiter=',')  # Read the data in from the text file
    return y_data  # Return the data array
    # This function returns the array of just the y values

  def getXTimeFromFile(self):
    x_time_path = data_folder_path + "TrainingData/" + self.xTimeName() # Get the path to the x_data file 
    x_time = np.genfromtxt(x_time_path, delimiter=',')  # Read the data in from the text file
    return x_time  # Return the data array
    # This function returns the array of just the x time values

  def getYTimeFromFile(self):
    y_time_path = data_folder_path + "TrainingData/" + self.yTimeName() # Get the path to the x_data file 
    y_time = np.genfromtxt(y_time_path, delimiter=',')  # Read the data in from the text file
    return y_time  # Return the data array
    # This function returns the array of just the y time values




This next code block creates a list of sessions with all the sessions included in the training data

In [ ]:
# Code block created by Caleb:
# Goal is to create list of each session, with a unique session id (session id made up of subject number and session number)

list_of_sessions = []

for file_name in list_of_files:
  # Extract subject and session number from file_name:
  new_subject_number = int(file_name[8:11]) # Since all provided subject and session numbers are single digit, we can 
  new_session_number = int(file_name[12:14]) #   pick them out of the file name by grabbing an exact character number.
  


  # Check if I should add this session to the list of sessions
  should_add_session = True # Create a variable that by default should add the session
  for session in list_of_sessions: # Loop through the sessions in the list of sessions (list I may need to add it to)
    comp_subject_number = session.subject_number # Get both new session and subject number
    comp_session_number = session.session_number #   save them as comp_
    if (comp_session_number == new_session_number) and (comp_subject_number == new_subject_number): # If comp session and subject are equal to new subject and session
      should_add_session = False # In this case, that would mean that I should /not/ add the new session

  if should_add_session: # If I should add the session
    newSession = Session(new_subject_number, new_session_number) # Initialize the new session object with the new subject and session values
    list_of_sessions.append(newSession) # Append the new session to the list of sessions

# Now list_of_sessions has a list of Session objects for each of the sessions
print(np.asarray(list_of_sessions).shape)
print(list_of_sessions[28].subject_number)
print(list_of_sessions[28].session_number)
# Print out the list of the sessions in list_of_sessions:
# for session in list_of_sessions:
#   newStr = "Subject: " + str(session.subject_number) + ". Session: " + str(session.session_number)
#   print(newStr) 


(29,)
8
1


This next code block pads the data. This means step two is now complete. (Steps can be seen in this photo: https://drive.google.com/file/d/1dQfBs5h9Dca7GaQmbXdu0xfJZGLYvUru/view?usp=sharing)

In [ ]:
# Code block created by Caleb
# Goal: Create a function that will take in an array of size [(length of file) x 6] and pad it with zeros for image size

# This function pads with image width - 1 rows of zeros
def padData(data, image_width):
  dim = [image_width - 1,6] # The dimensions of the zeros will be the width of the image and six wide
  zeroArray = np.zeros(dim) # Create an array with zeros at the beginning of the correct size
  return np.concatenate((zeroArray, data)) # Concatonate the two arrays together with the zeros at the beginning

# Print out the data for a small data array to see where the zeros go
data = np.array([[1, 2, 3, 4, 5, 6], [3, 4, 5, 6, 7, 5]])
paddedData = padData(data, 3) 
print(data)

[[1 2 3 4 5 6]
 [3 4 5 6 7 5]]


Now, this next block needs to create "images" that have a certain definable width, and those will be of dimension [image_width x 6].

There should be 40\*(session_length) number of images, so the output of this will be a three-dimensional array [(40\*session_length x image_width x 6]



In [ ]:
# Code block created by Caleb
# Goal: Step three of the whiteboard photo, so create "images" from padded data

# Work on a variable image stride length

# This function takes in the padded data (an array  [(image_width + 40*session lenght).x 6] ) and an integer of the width of an image
def createImages(paddedData, image_width, image_stride):
  assert image_stride > 0, "Error: Image stride length must be at least 1!"
  all_images = [] # Create an empty array that will eventually hold all the images for this session's data
  possible_num_images = paddedData.shape[0] - image_width + 1  # The number of images I should create is padded_data - (image_width - 1)
  # print("Number of possible images: " + str(possible_num_images))  
  for i in range(0, possible_num_images, image_stride): # Loop through i for each of the images I need to create
    all_images.append(paddedData[i:(i+image_width)])  # Grab out the array from the padded data equal to the image_width and starting at i, and append it to all images
  # print(len(all_images))
  return np.asarray(all_images)




# This example uses the 'data' array from the code block above to pass into this function to see if it successfully creates two images
#   One should have two rows of zeros (because it gets all the padding) and the next should have one row of zeros
images = createImages(paddedData, 3, 1)
print(images)

[[[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [1. 2. 3. 4. 5. 6.]]

 [[0. 0. 0. 0. 0. 0.]
  [1. 2. 3. 4. 5. 6.]
  [3. 4. 5. 6. 7. 5.]]]


This next code block will take in an x_time values, y_time values, and y values, and outputs a new list of y values that line up to time with the x_time places.

In order to do this, it will take all x_time values, and for each value of y_time that is closest to the x_time it will add the y value for that y_time value to a new list. It then returns that new list.

Note: we asked Dr. Lobaton what rate the camera took frames, and he said that the camera was 10hz. The students tried for the accuracy of the labels to be per frame (so 10hz what we were given), but he said we shouldn't consider the accuracy to be better than 200ms.

In [ ]:
# Block created by Caleb
# Goal: make a function that "upsamples" the y values. It will therefore get the closest-labelled y value for each x_time measurement value

# This function takes in the x_time values and the y_time values, along with the y values
# For each of the x_time values, it finds the index of the y_time value that is closest.
# Then, it appends to the new y list the value of y at the index where the closest y_time value was found
def extrapolatedYs(x_time, y_time, y, image_stride_size):
  transformedY = [] #Empty list for the newly transformed y values
  y_time_array = np.asarray(y_time) # Create the y_time values as an array
  assert image_stride_size > 0, "Error: Step size must be at least one!"
  for x_t in range(0, len(x_time), image_stride_size): # Loop over every x_time value
    index = (np.abs(y_time_array - x_time[x_t])).argmin() # I found this online to get the index of the closest value from the y_time arrays
    transformedY.append(y[index]) # Add the y value at that index to the transformed y's

  
  return np.asarray(transformedY)
# This example shows how to use this function above
my_x_time = [0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325]
my_y_time = [          0.02,                     .12,                     .22,        .32]
my_y =  [1,     4,   3,   2]
my_image_step_size = 3

new_y = extrapolatedYs(my_x_time, my_y_time, my_y, my_image_step_size)
print(new_y)

[1 4 4 3 2]


I figured this is a good point to create just an example block for how to use the code that is above only.

The final output at the bottom are all the images from a single session


In [ ]:
# I will use the first session for this example:
example_session = list_of_sessions[0]

example_x_data = example_session.getXDataFromFile()
example_x_time = example_session.getXTimeFromFile()
example_y_data = example_session.getYDataFromFile()
example_y_time = example_session.getYTimeFromFile()

example_image_stride = 2

# Now I will extrapolate out the y's:
example_extrapolated_y = extrapolatedYs(example_x_time, example_y_time, example_y_data, example_image_stride)

# Now the x data and the extrapolate_y have the correct sizes, so y is the predictions we want
print("New y size, and then x data shape")
print(example_extrapolated_y.shape)
print(example_x_data.shape)
print("- - -")

# Now I will show the examples of transforming the x values into images. 
example_image_width = 60 # You can play around with the width of the image
example_x_padded = padData(example_x_data, example_image_width)
print("With padding:")
print(example_x_padded.shape)
print("- - -")

example_x_images = createImages(example_x_padded, example_image_width, example_image_stride)
print("Image Stride Size: " + str(example_image_stride))
print("Shape of images, so should be back down to number of x values / image stride size, with image_size x 6 secondary dimensions")
print(example_x_images.shape)
print("- - -")
print("Printing first images")
# print(example_x_images[0])
# print(example_x_images[1])

# So the data we're treating just like images for classification are:
# example_x_images          as the images
# example_extrapolated_y    as the labels

New y size, and then x data shape
(18945,)
(37890, 6)
- - -
With padding:
(37949, 6)
- - -
Image Stride Size: 2
Shape of images, so should be back down to number of x values / image stride size, with image_size x 6 secondary dimensions
(18945, 60, 6)
- - -
Printing first images


The code block below will solve Step 4 from the picture, which is splitting the images up into Sets. 

Edit Mar 23rd:
After talking with Dr. Lobaton, Nathan and I found out that the y values were created from a camera frame rate of y, so I've already created a function above that extrapolates y values out to have the same number of x values

In [ ]:
# Code block created by Caleb

# Becuase a set needs to have both the images and the associated labels (y values)
#   I was thinking of creating a set class that just has arrays of the images and y values, that way we can keep the two types of data together within the sets
class Set:
  def __init__(self, images, labels):
    self.images = images  # Initialize subject number
    self.labels = labels  # Initialize session number
  

In [ ]:
# Code block created by Caleb
# Goal: create a function that creates the sets

# Pass in the set length. This is the number of images per set
# Also pass in image_width
# images and labels (y_data) need to have the same length 
# The dimensions of a set are 
# Returns an array of the sets from that data

# This is the function I am most concerned is not correct, so handle with care, lol


def createSets(images, labels, set_length, image_width):
  list_of_sets = []
  num_of_images = images.shape[0] # These are how many images there are
  # print("Number of images in set creation function: " + str(num_of_images))
  num_of_labels = labels.size
  assert(num_of_images == num_of_labels),"The number of images does not match the number of correponding labels. This error is in createSets."


  i = 0 # Counter for which image about to get. This starts at the zeroth image I am going to put into a set first
  while (num_of_images - i) >= set_length: # Loop until can't make any more sets. If there are not enough images left to make another whole set, stop making tests
      
    new_set = Set(images[i:(i + set_length)], labels[i:(i + set_length)]) # Create a new set with as many images as are supposed to be in the set. has both images and labels
    list_of_sets.append(new_set) # Append this new set to the list of sets

    # Now I need to increment i:
    i = i + set_length  # Increment past what I just added to the set, which is the set size
    # Increment i to ignore image width - 1 more images as the buffer between sets
    ignored_images_for_padding = image_width - 1 
    i = i + ignored_images_for_padding
    
  return np.asarray(list_of_sets)
  


# # This example shows how to use this function above
# my_x_time = [0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275, 0.3]
# my_x_data = [[1, 2, 3, 4, 5, 6], [2, 3, 4, 5, 6, 7], [3, 4, 5, 6, 7, 8], [1, 2, 2, 4, 5, 6], [9, 2, 3, 4, 5, 6], [1, 3, 3, 2, 5, 6], [1, 2, 4, 4, 7, 6], [1, 2, 2, 4, 9, 9], [4, 4, 4, 4, 4, 4], [7, 2, 2, 4, 5, 6], [3, 3, 3, 4, 3, 6], [1, 2, 2, 4, 2, 2], [10, 22, 33, 24, 50, 6] ]
# my_y_time = [          0.02,                     .12,                     .22,        .32]
# my_y =  [1,     4,   3,   2]
# my_extrapolated_y = extrapolatedYs(my_x_time, my_y_time, my_y)
# my_image_width = 3 # You can play around with the width of the image
# my_x_padded = padData(my_x_data, my_image_width)
# my_x_images = createImages(my_x_padded, my_image_width)

# my_set_size = 4

# my_sets = createSets(my_x_images, my_extrapolated_y, my_set_size, my_image_width)

#print(my_sets.shape)

#for singleSet in my_sets:
#  print(singleSet.images)
#  print(singleSet.labels)

# example_x_images
# example_image_width
# example_extrapolated_y
example_set_size = 800
example_set_step = 2

example_sets = createSets(example_x_images, example_extrapolated_y, example_set_size, example_image_width)
print(example_x_images.shape)
print(example_image_width)
print(example_sets.shape)
#print(example_sets[0].images)
#print(example_sets[1].images)




(18945, 60, 6)
60
(22,)


Now we need a function that takes in the sets, and percentages for train and validation (test will be 100% - train - validation) and splits the sets up randomly in that order.

I will take the floor of the (number of sets)\*percentage. AKA, if there are 44 sets, and we want 40% test data I will make 17 test sets since 44*.4 = 17.6. Then, I will do the same floor function on the validation amount, and then take the remaining ones for testing. So that means there will most likely be between 1 and 2 more test sets than we "want" based on the percentages we chose.

I also made it so that we can pass in the seed so that we can control that and always get the same sets as we go along.

In [ ]:
# Code block created by Caleb

def splitUpSets(list_of_sets, percentage_train, percentage_val, seed):
  training_sets = [] # Create empty lists for the lists of sets that will be returned
  validation_sets = []
  testing_sets = []
  num_of_sets = list_of_sets.size # Get the total number of sets passed into the function
  num_of_train_sets = math.floor(num_of_sets*percentage_train/100) # Take the floor of the percentage of each
  num_of_val_sets = math.floor(num_of_sets*percentage_val/100)

  print("Num sets: " + str(num_of_sets))
  print("Num Training Sets: " + str(num_of_train_sets))
  print("Num Validation Sets: " + str(num_of_val_sets))
  print("Num Test Sets: " + str(num_of_sets - num_of_train_sets - num_of_val_sets))
  random.seed(seed) # Set the random seed based upon the seed passed in

  for i in range(num_of_train_sets): # Loop over the number of train sets
    index_to_remove = random.randrange(0, num_of_sets, 1) # Find a random index to remove
    training_sets.append(list_of_sets[index_to_remove]) # Append the set to training_sets
    list_of_sets = np.delete(list_of_sets, index_to_remove)  # Remove the set from the master list
    # Decrement number of sets
    num_of_sets -= 1

  for i in range(num_of_val_sets): # Loop over the number of train sets
    index_to_remove = random.randrange(0, num_of_sets, 1) # Find a random index to remove
    validation_sets.append(list_of_sets[index_to_remove]) # Append the set to validation_sets
    list_of_sets = np.delete(list_of_sets, index_to_remove) # Remove the set from the master list
    # Decrement number of sets
    num_of_sets -= 1
  
  testing_sets = list_of_sets # Training gets whatever is left

  return np.asarray(training_sets), np.asarray(validation_sets), np.asarray(testing_sets) # Return arrays of the three types

    


example_train, example_val, example_test = splitUpSets(example_sets, 40, 30, 10)

print(example_train[3].labels)
print(example_val[1].labels)
print(example_test[3].labels)

Num sets: 22
Num Training Sets: 8
Num Validation Sets: 6
Num Test Sets: 8
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2

In [ ]:
'''
Testing block for set creation
'''

print("Processing Subject: " + str(session.subject_number) + ". Session: " + str(session.session_number))

   # I will use the first session for this example:
example_session = list_of_sessions[0]

example_x_data = example_session.getXDataFromFile()
example_x_time = example_session.getXTimeFromFile()
example_y_data = example_session.getYDataFromFile()
example_y_time = example_session.getYTimeFromFile()

example_image_stride = 4

# Now I will extrapolate out the y's:
example_extrapolated_y = extrapolatedYs(example_x_time, example_y_time, example_y_data, example_image_stride)

# Now the x data and the extrapolate_y have the correct sizes, so y is the predictions we want
print("New y size, and then x data shape")
print(example_extrapolated_y.shape)
print(example_x_data.shape)
print("- - -")

# Now I will show the examples of transforming the x values into images. 
example_image_width = 60 # You can play around with the width of the image
example_x_padded = padData(example_x_data, example_image_width)
print("With padding:")
print(example_x_padded.shape)
print("- - -")

example_x_images = createImages(example_x_padded, example_image_width, example_image_stride)
print("Image Stride Size: " + str(example_image_stride))
print("Shape of images, so should be back down to number of x values / image stride size, with image_size x 6 secondary dimensions")
print(example_x_images.shape)
print("- - -")

# print(example_x_images[0])
# print(example_x_images[1])

# So the data we're treating just like images for classification are:
# example_x_images          as the images
# example_extrapolated_y    as the labels

example_set_size = 800
example_set_step = 1
print(example_x_images.shape)
print(example_image_width)
example_sets = createSets(example_x_images, example_extrapolated_y, example_set_size, example_image_width)
print(example_sets.shape)

Processing Subject: 8. Session: 1
New y size, and then x data shape
(9473,)
(37890, 6)
- - -
With padding:
(37949, 6)
- - -
Image Stride Size: 4
Shape of images, so should be back down to number of x values / image stride size, with image_size x 6 secondary dimensions
(9473, 60, 6)
- - -
(9473, 60, 6)
60
(11,)


Now process all the data and make HUGE arrays with all of the data.

Pass in all the hyperparameters we made

In [ ]:
# Code block created by Caleb

# Inputs:
#   list_of_sessions
#   image_width (in seconds)
#   set_length (in seconds)
#   training percentage
#   validation percentage
#   random number seed 
def getDataForModel(list_of_sessions, image_width, image_stride_size, set_length, train_percentage, val_percentage, seed):
  image_width_number = int(image_width * 40) # This is the conversion rate between image_width in seconds and image_width the number of samples in an image
  set_length_number = int(set_length * 40) # Convert to number of images in a set
  assert image_width > 0, "Error: Image width must be at least 1!"
  assert image_stride_size > 0, "Error: Image width must be at least 1!"
  x_train = [] # Initialize the lists to zero
  y_train = []
  x_val = []
  y_val = []
  x_test = []
  y_test = []

  # Create an array of the random seed values to pass in to the functions
  random.seed(seed) # Seed the random number generator with the value passed in 
  random_seeds = []
  # For each of the lists of sessions, append a new random number to the random seeds
  for i in range(len(list_of_sessions)):
    random_seeds.append(random.randint(-999999999, 999999999)) # I chose these bounds completely arbitrariliy

  index_of_random = 0 # I need a list to access the random numbers I just created

  # Iterate over each sessoin in the list of sessions:
  for session in list_of_sessions:
    #strToPrint = "Processing Subject: " + str(session.subject_number) + ". Session: " + str(session.session_number)
    print("Processing Subject: " + str(session.subject_number) + ". Session: " + str(session.session_number))

    # Take a session and get a list of sets

    x_data = session.getXDataFromFile() # Get all the data for that session
    x_time = session.getXTimeFromFile()
    y_data = session.getYDataFromFile()
    y_time = session.getYTimeFromFile()
    

    extrapolated_y = extrapolatedYs(x_time, y_time, y_data, image_stride_size) # Now I will extrapolate out the y's:
    

    x_padded = padData(x_data, image_width_number) # Pad the x values
    x_images = createImages(x_padded, image_width_number, image_stride_size) # Then create images
    

    print("Shape of x_images:")
    print(np.asarray(x_images).shape)
    print("Shape of extrapolated y:")
    print(np.asarray(extrapolated_y).shape)
    
    # Then create sets
    list_of_sets = createSets(x_images, extrapolated_y, set_length_number, image_width_number)
    
    print("Number of sets:")
    print(np.asarray(list_of_sets).shape)

    # Get the three sets from this one
    train, val, test = splitUpSets(list_of_sets, train_percentage, val_percentage, random_seeds[index_of_random])

    index_of_random += 1 # Make sure to increment index of random numbers since just used one of them

    # Loop over training data, and append the images and labels
    for individualSet in train:
      for image in individualSet.images:
        x_train.append(image)

      for label in individualSet.labels:
        y_train.append(label)


    # Loop over validation data, and append the images and labels
    for individualSet in val:
      for image in individualSet.images:
        x_val.append(image)

      for label in individualSet.labels:
        y_val.append(label)


    # Loop over testing data, and append the images and labels
    for individualSet in test:
      for image in individualSet.images:
        x_test.append(image)

      for label in individualSet.labels:
        y_test.append(label)

    assert(np.asarray(x_train).shape[0] == len(y_train)),"Wrong "
    assert(np.asarray(x_val).shape[0] == len(y_val)),"Wrong "
    assert(np.asarray(x_test).shape[0] == len(y_test)),"Wrong "

  print("Done processing all sessions!")
  return np.asarray(x_train), np.asarray(y_train), np.asarray(x_val), np.asarray(y_val), np.asarray(x_test), np.asarray(y_test)

#Yt_train = Yt_train.type(torch.LongTensor)
# This next line calls the function to get the data for the model with the parameters passed in.
# Remember that image width and set length are in seconds
# getDataForModel(list_of_sessions, image_width, image_step_size, set_length, train_percentage, val_percentage, seed)
x_train, y_train, x_val, y_val, x_test, y_test = getDataForModel(list_of_sessions, 1.5, 4, 20, 70, 20, 1)


Processing Subject: 1. Session: 1
Shape of x_images:
(9473, 60, 6)
Shape of extrapolated y:
(9473,)
Number of sets:
(11,)
Num sets: 11
Num Training Sets: 7
Num Validation Sets: 2
Num Test Sets: 2
Processing Subject: 1. Session: 2
Shape of x_images:
(17543, 60, 6)
Shape of extrapolated y:
(17543,)
Number of sets:
(20,)
Num sets: 20
Num Training Sets: 14
Num Validation Sets: 4
Num Test Sets: 2
Processing Subject: 1. Session: 3
Shape of x_images:
(10787, 60, 6)
Shape of extrapolated y:
(10787,)
Number of sets:
(12,)
Num sets: 12
Num Training Sets: 8
Num Validation Sets: 2
Num Test Sets: 2
Processing Subject: 1. Session: 4
Shape of x_images:
(13739, 60, 6)
Shape of extrapolated y:
(13739,)
Number of sets:
(16,)
Num sets: 16
Num Training Sets: 11
Num Validation Sets: 3
Num Test Sets: 2
Processing Subject: 1. Session: 5
Shape of x_images:
(14781, 60, 6)
Shape of extrapolated y:
(14781,)
Number of sets:
(17,)
Num sets: 17
Num Training Sets: 11
Num Validation Sets: 3
Num Test Sets: 3
Processin

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)
print(type(y_val[0]))


(206400, 60, 6)
(206400,)
(52800, 60, 6)
(52800,)
(47200, 60, 6)
(47200,)
<class 'numpy.float64'>


So I am very hesitant to say this because of how unbelievably new I am to Python, but I *think* I may have actually gotten all the data augmentation done. Those six arrays which the sizes are printed above should contain all the data we need to feed into our convolutional neural network.

Please take this with a grain of salt, because I probably have many, many bugs, but I did everything I could think of for it.

The below code block was made by Trenton and prints the data for Subject 1, Session 1:

In [ ]:

# # pulling data from 
# path_to_trainingdata = os.path.abspath(data_folder_path + type_of_data)
# subject_001_01__x = os.path.join(path_to_trainingdata,'subject_003_01__x.csv' )
# subject_001_01__x_time = os.path.join(path_to_trainingdata, 'subject_003_01__x_time.csv')
# subject_001_01__y = os.path.join(path_to_trainingdata,'subject_003_01__y.csv' )
# subject_001_01__y_time = os.path.join(path_to_trainingdata,'subject_003_01__y_time.csv' )
# # pull data from the file
# imu_x = np.genfromtxt(subject_001_01__x,delimiter=',')
# imu_y = np.genfromtxt(subject_001_01__y, delimiter='\n')
# imu_x_time = np.genfromtxt(subject_001_01__x_time, delimiter='\n')
# imu_y_time = np.genfromtxt(subject_001_01__y_time, delimiter='\n')
# #create plot
# fig, axes = plt.subplots(2, 1)

# axes[0].plot(imu_x_time, imu_x)
# axes[1].plot(imu_y_time, imu_y)

CODE BASED ON HOMEWORK 5

In [ ]:
import os
import torch
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import sklearn.utils

class CustomImageDataset(Dataset):
    def __init__(self, x_data, y_data, img_dir=None, transform=None, target_transform=None):
        self.img_labels = y_data
        self.img_dir = img_dir
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = self.x_data[idx]
        label = self.y_data[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


print("Shuffling within sets!") # Trying to improve performance by shuffling the images within each set
# np.random.shuffle(x_train)
# np.random.shuffle(x_val)
# np.random.shuffle(x_test)


# def unison_shuffled_copies(a, b):
#     assert len(x_train) == len(y_train)
#     p = np.random.permutation(len(a))
#     return a[p], b[p]

#x_train_shuffled, y_train_shuffled = unison_shuffled_copies(x_train, y_train)
x_train_shuffled, y_train_shuffled = sklearn.utils.shuffle(x_train, y_train)



our_transform = transforms.Compose([
    # Converting RGB [0,255] to Tensor [0,1]
    transforms.ToTensor(),
    ])

batch_size = 10;



def createNewSubSet(num_set, main_set_x, main_set_y):
  subset_x = []
  subset_y = []
  for i in range(num_set): # Loop over the number of sets
      index_to_add = random.randrange(0, len(main_set_x), 1) # Find a random index to remove
      subset_x.append(main_set_x[index_to_add]) # Append the set to training_sets
      subset_y.append(main_set_y[index_to_add])
  return np.asarray(subset_x), np.asarray(subset_y)



x_train_len = len(x_train_shuffled)
x_val_len = len(x_val) 
x_test_len = len(x_test)

# small_x_train, small_y_train = createNewSubSet(x_train_len, x_train_shuffled, y_train_shuffled)
# small_x_val, small_y_val = createNewSubSet(x_val_len, x_val, y_val)
# small_x_test, small_y_test = createNewSubSet(x_test_len, x_test, y_test)



x_train_re = np.reshape(x_train_shuffled, [x_train_shuffled.shape[0], 6, 60, 1])
x_val_re = np.reshape(x_val, [x_val.shape[0], 6, 60, 1])
x_test_re = np.reshape(x_test, [x_test.shape[0], 6, 60, 1])

y_train_t = torch.tensor(y_train_shuffled).type(torch.LongTensor) 
y_val_t = torch.tensor(y_val).type(torch.LongTensor) 
y_test_t = torch.tensor(y_test).type(torch.LongTensor) 



print(y_train_t[0].dtype)

train_data = CustomImageDataset(x_train_re, y_train_t, transform=our_transform)
valid_data = CustomImageDataset(x_val_re, y_val_t, transform=our_transform)
test_data = CustomImageDataset(x_test_re, y_test_t, transform=our_transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)


_iter = iter(train_loader)
print(type(_iter))
image_batch, index = next(_iter)
print(image_batch.shape)


Shuffling within sets!
torch.int64
<class 'torch.utils.data.dataloader._SingleProcessDataLoaderIter'>
torch.Size([10, 1, 6, 60])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Defining the CNN architecture
class Net(nn.Module):
  def __init__(self, ):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, 5, padding=2, padding_mode='zeros', dtype=float)
   ## nn.MaxPool2d()
    self.conv2 = nn.Conv2d(6, 16, 5, padding=2, padding_mode='zeros', dtype=float)
    self.drop2 = nn.Dropout2d(0.1)
    self.drop1 = nn.Dropout(0.1)
    self.batchNorm1 = nn.BatchNorm2d(6, dtype = float)
    self.batchNorm2 = nn.BatchNorm2d(16, dtype = float)
    self.batchNorm3 = nn.BatchNorm1d(800, dtype = float)
    self.batchNorm4 = nn.BatchNorm1d(200, dtype = float)
    self.batchNorm5 = nn.BatchNorm1d(50, dtype = float)
    self.fc1 = nn.Linear(16 * 60 * 6, 800, dtype=float)
    self.fc2 = nn.Linear(800, 200, dtype=float) 
    self.fc3 = nn.Linear(200, 50, dtype=float)
    self.fc4 = nn.Linear(50, 4, dtype=float)
  def forward(self, x):
    x = self.drop2(F.relu(self.batchNorm1(self.conv1(x))))
    x = self.drop2(F.relu(self.batchNorm2(self.conv2(x))))
    x = x.view(-1, 16 * 60 * 6) # At this point the feature map is 5 x 5 x 16
    x = self.drop1(F.relu(self.batchNorm3(self.fc1(x))))
    x = self.drop1(F.relu(self.batchNorm4(self.fc2(x))))
    x = self.drop1(F.relu(self.batchNorm5(self.fc3(x))))
    x = self.fc4(x)
    return x

# Create a complete CNN
model = Net()
print(model)

# Move tensors to GPU if CUDA is available
flag_cuda = torch.cuda.is_available()
if flag_cuda:
  model.cuda()

# Specifying the loss function
criterion = nn.CrossEntropyLoss()

# Specify optimizer
optimizer = optim.Adam(model.parameters(), lr=.001)
#optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)




Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (drop2): Dropout2d(p=0.1, inplace=False)
  (drop1): Dropout(p=0.1, inplace=False)
  (batchNorm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchNorm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchNorm3): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchNorm4): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchNorm5): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=5760, out_features=800, bias=True)
  (fc2): Linear(in_features=800, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=4, bias=True)
)


In [ ]:
import matplotlib.pyplot as plt

# Specifying the number of epochs
n_epochs = 8

def trainNet(model,criterion,optimizer,n_epochs,flag_cuda):
  # Unpacking the number of epochs to train the model
  epochs_list = [*range(1,n_epochs+1)]

  # List to store loss to visualize
  train_losslist = []
  valid_losslist = []
  valid_loss_min = np.Inf # track change in validation loss

  for epoch in epochs_list:
      # Keeping track of training and validation loss
      train_loss = 0.0
      valid_loss = 0.0
      
      ######################
      # Training the model #
      ######################
      model.train()
      for data, target in train_loader:
          # Moving tensors to GPU if CUDA is available
          if flag_cuda:
              data, target = data.cuda(), target.cuda()
          # Clearing the gradients of all optimized variables
          optimizer.zero_grad()
          # Forward pass: Computing predicted outputs
          output = model(data)
          # Calculating the batch loss
          loss = criterion(output, target)
          # Backward pass: compute gradient of loss with respect to parameters
          loss.backward()
          # Perform a single optimization step (parameter update)
          optimizer.step()
          # Update training loss
          train_loss += loss.item()*data.size(0)
          
      ########################    
      # Validating the model #
      ########################
      model.eval()
      for data, target in valid_loader:
          # Moving tensors to GPU if CUDA is available
          if flag_cuda:
              data, target = data.cuda(), target.cuda()
          output = model(data)
          loss = criterion(output, target)
          valid_loss += loss.item()*data.size(0)
      
      # Calculating average losses
      train_loss = train_loss/x_train_len
      valid_loss = valid_loss/x_val_len
      train_losslist.append(train_loss)
      valid_losslist.append(valid_loss)
          
      # Printing training/validation statistics 
      print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch, train_loss, valid_loss))
      
      # Saving model if validation loss has decreased
      if valid_loss <= valid_loss_min:
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
              valid_loss_min,valid_loss))
          torch.save(model.state_dict(), 'model_cifar.pt')
          valid_loss_min = valid_loss
        
  return epochs_list, train_losslist, valid_losslist

# Executing the training
epochs_list, train_losslist, valid_losslist = trainNet(
    model,criterion,optimizer,n_epochs,flag_cuda)

# Loading the best model
model.load_state_dict(torch.load('model_cifar.pt'))

# Plotting the learning curves
plt.plot(epochs_list, train_losslist, epochs_list, valid_losslist)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(['Training','Validation'])
plt.title("Performance of Baseline Model")
plt.show()

Epoch: 1 	Training Loss: 0.814100 	Validation Loss: 0.846385
Validation loss decreased (inf --> 0.846385).  Saving model ...
Epoch: 2 	Training Loss: 0.807735 	Validation Loss: 0.843308
Validation loss decreased (0.846385 --> 0.843308).  Saving model ...


KeyboardInterrupt: ignored

In [ ]:
def saveModel(model, filename):
  competition_path = "/content/drive/Shareddrives/Neural Nets/Competition/Phase 1/Code/Models" #path to data folder in drive
  model_file_path = os.path.join(competition_path, filename+".pt")  #joing path and adding .npy to filename passed in
  torch.save(model.state_dict(), model_file_path)

saveModel(model, "nathan_trying_to_fix_underfitting")

In [ ]:

classes = [0, 1, 2, 3]

def assessNet(model,criterion):
  # Tracking test loss and accuracy
  test_loss = 0.0
  class_correct = list(0. for i in range(len(classes)))
  class_total = list(0. for i in range(len(classes)))

  # Setting model to evaluate
  model.eval()

  # Iterating over batches of test data
  for data, target in test_loader:
      # Obtaining predictions and loss
      if flag_cuda:
          data, target = data.cuda(), target.cuda()
      output = model(data)
      loss = criterion(output, target)
      test_loss += loss.item()*data.size(0)

      # Converting output probabilities to predicted class
      _, pred = torch.max(output, 1)    
      # Comparing predictions to true label
      correct_tensor = pred.eq(target.data.view_as(pred))
      correct = np.squeeze(correct_tensor.numpy()) if not flag_cuda else np.squeeze(correct_tensor.cpu().numpy())
      # Calculating test accuracy for each object class
      for i in range(batch_size):
          label = target.data[i]
          class_correct[label] += correct[i].item()
          class_total[label] += 1

  # Computing the average test loss
  test_loss = test_loss/len(test_loader.dataset)
  print('Test Loss: {:.6f}\n'.format(test_loss))

  # Computing the class accuracies
  for i in range(4):
      if class_total[i] > 0:
          print('Test Accuracy of %10s: %2d%% (%2d/%2d)' % (
              classes[i], 100 * class_correct[i] / class_total[i],
              np.sum(class_correct[i]), np.sum(class_total[i])))
      else:
          print('Test Accuracy of %10s: N/A (no training examples)' % (classes[i]))

  # Computing the overall accuracy
  print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
      100. * np.sum(class_correct) / np.sum(class_total),
      np.sum(class_correct), np.sum(class_total)))
  
  return

# Executing the assessment
assessNet(model,criterion)

Test Loss: 0.731103

Test Accuracy of          0: 100% (36898/36898)
Test Accuracy of          1:  0% ( 0/1635)
Test Accuracy of          2:  0% ( 0/2332)
Test Accuracy of          3:  0% ( 0/6335)

Test Accuracy (Overall): 78% (36898/47200)
